In [2]:
#import libraries
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution
from edge_extractor import EdgeExtracter
from medusa_tools import *
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [3]:
#setup example
bd.projects.set_current("test_abc")


In [4]:
bd.Database('temporalis-bio').write({
    ('temporalis-bio', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-bio', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
})

bd.Database('background_2022').write({
    ('background_2022', 'C'): {
    'name': 'process C',
    'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2022', 'C'),
            },
            {
                'amount': 1,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },  ]},
    ('background_2022', 'D'): {
    'name': 'process D',
    'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2022', 'D'),
            },
            {
                'amount': 1,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
       
        ]},
}
        )

bd.Database('background_2020').write({
    ('background_2020', 'C'): {
    'name': 'process C',
    'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'C'),
            },
            {
                'amount': 2,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },  ]},
    
    ('background_2020', 'D'): {
    'name': 'process D',
    'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'D'),
            },
            {
                'amount': 3,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
       
        ]},
}
        )

bd.Database('foreground').write({
    ('foreground', 'A'): {
        'name': 'process A',
        'temporal_distribution': TemporalDistribution(
                    np.array(['2023-01-04T10:30:58'], dtype='datetime64[s]'),
                    np.array([1])),
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'A'),
                'temporal_distribution': TemporalDistribution(
                    np.array(['2023-01-04T10:30:58'], dtype='datetime64[s]'),
                    np.array([1]))
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('foreground', 'B'),
                'temporal_distribution': # e.g. because some hydrogen was stored in the meantime
                    easy_timedelta_distribution(
                    start=-2,
                    end=0, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=2,
                ),
            },
        ]
    },
    ('foreground', 'B'):
    {
        "name": "process B",
        "exchanges": [
             {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2022', 'C'),
             }
        ]

    },
})

100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 


In [5]:
for act in bd.Database("foreground"):
    print(act)
    if "temporal_distribution" in act:
        print(act["temporal_distribution"].amount)
        print(act["temporal_distribution"].date)
        

for act in bd.Database("background_2022"):
    print(act)
    if "temporal_distribution" in act:
        print(act["temporal_distribution"].amount)
        print(act["temporal_distribution"].date)

'process A' (None, None, None)
[1.]
['2023-01-04T10:30:58']
'process B' (None, None, None)
'process D' (None, None, None)
'process C' (None, None, None)


The TD on the exchange between B and A is not stored at the activity (see print above) but at the level of exchanges (see print below)

In [6]:
for act in bd.Database("foreground"):
    for exc in act.exchanges():
        if "temporal_distribution" in exc:
            print(f"There is a temporal information for {exc}")
            print(exc["temporal_distribution"].amount)
            print(exc["temporal_distribution"].date)

There is a temporal information for Exchange: 1 None 'process A' (None, None, None) to 'process A' (None, None, None)>
[1.]
['2023-01-04T10:30:58']
There is a temporal information for Exchange: 1 None 'process B' (None, None, None) to 'process A' (None, None, None)>
[0.5 0.5]
[-63113904         0]


In [7]:
bd.Method(("GWP", "example")).write([
    (('temporalis-bio', "CO2"), 1),
    (('temporalis-bio', "CH4"), 25),
])

In [8]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

In [9]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 1.0


Medusa LCA

In [10]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2023')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

In [11]:
eelca = EdgeExtracter(slca, edge_filter_function=filter_function)
timeline = eelca.build_edge_timeline()
timeline


Starting graph traversal
Calculation count: 2


[Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=-1, producer=23),
 Edge(distribution=TemporalDistribution instance with 2 values and total: 1, leaf=False, consumer=23, producer=24),
 Edge(distribution=TemporalDistribution instance with 2 values and total: 1, leaf=False, consumer=24, producer=19)]

In [12]:
database_date_dict = {
            datetime.strptime("2022", "%Y"): 'background_2022',
            datetime.strptime("2020", "%Y"): 'background_2020',
        }
print(database_date_dict.keys())
timeline_df = create_grouped_edge_dataframe(timeline, database_date_dict.keys(), interpolation_type="linear")
timeline_df

dict_keys([datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2020, 1, 1, 0, 0)])


date  year  producer producer_name  consumer consumer_name amount  \
0 2022-01-01  2022        19     process C        24     process B    0.5   
1 2022-01-01  2022        24     process B        23     process A    0.5   
2 2024-01-01  2024        19     process C        24     process B    0.5   
3 2024-01-01  2024        23     process A        -1            -1    1.0   
4 2024-01-01  2024        24     process B        23     process A    0.5   

  interpolation_weights  
0             {2022: 1}  
1             {2022: 1}  
2             {2022: 1}  
3             {2022: 1}  
4             {2022: 1}

In [ ]:
from datetime import datetime, timedelta

def find_closest_dates(reference_date, dates_list):
    if not dates_list:
        print("The provided list of dates is empty.")
        return None, None

    closest_lower = None
    closest_higher = None

    for date in dates_list:
        if date < reference_date:
            if closest_lower is None or reference_date - date < reference_date - closest_lower:
                closest_lower = date
        elif date > reference_date:
            if closest_higher is None or date - reference_date < closest_higher - reference_date:
                closest_higher = date

    if closest_lower is None:
        print("Warning: Reference date is higher than all provided dates.")
        closest_lower = closest_higher

    if closest_higher is None:
        print("Warning: Reference date is lower than all provided dates.")
        closest_higher = closest_lower

    return closest_lower, closest_higher

# Example usage:
reference_date = datetime(2024, 1, 1)
dates_list = [
    datetime(2023, 12, 31),
    datetime(2024, 1, 2),
    datetime(2024, 1, 5),
    datetime(2024, 1, 10),
]

closest_lower, closest_higher = find_closest_dates(reference_date, dates_list)

print("Reference Date:", reference_date)
print("Closest Lower Date:", closest_lower)
print("Closest Higher Date:", closest_higher)

In [45]:
demand_timing_dict = create_demand_timing_dict(timeline_df, demand)

dp = create_datapackage_from_edge_timeline(timeline_df, database_date_dict, demand_timing_dict)

In [46]:
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()
lca.score

1.249828884325804